In [1]:
%cd ..

/Users/chan/boostcamp-project/level2-nlp-generationfornlp-nlp-06-lv3


/Users/chan/boostcamp-project/level2-nlp-generationfornlp-nlp-06-lv3/.venv/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [ ]:
import os

import pandas as pd
from datasets import load_dataset
from dotenv import load_dotenv
from langchain.output_parsers import BooleanOutputParser, OutputFixingParser
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from tqdm import tqdm

from prompts import load_template

/Users/chan/boostcamp-project/level2-nlp-generationfornlp-nlp-06-lv3/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
load_dotenv()

PARAGRAPH = "paragraph"
QUESTION = "question"
CHOICES = "choices"
ANSWER = "answer"

VALID = "is_valid"

dataset = load_dataset("skt/kobest_v1", "boolq")

In [ ]:
train_df = pd.DataFrame(dataset["train"])
valid_df = pd.DataFrame(dataset["validation"])
test_df = pd.DataFrame(dataset["test"])

kobest_df = pd.concat([train_df, valid_df, test_df])
kobest_df.head()

,paragraph,question,label
0,로마 시대의 오리엔트의 범위는 제국 내에 동부 지방은 물론 제국 외부에 있는 다른 ...,"오리엔트는 인도와 중국, 일본을 이루는 광범위한 지역을 지칭하는 단어로 쓰인다.",1
1,비글을 키우려면 비글이 뛰어놀수 있는 넓은 놀이공간 등을 확보하고 있는 단독주택이 ...,비글은 넓고 뚤린 공간에서 키워야 한다.,1
2,타이완 요리의 특징은 토속 요리(일본 통치 전)에서 기름을 많이 사용하는 다른 지역...,타이완 요리는 다른 지역의 중국 요리처럼 기름을 많이 사용하는 것이다.,0
3,연하곤란은 음식물이 구강에서 식도로 넘어가는 과정에 문제가 생겨 음식을 원활히 혹은...,연하곤란이 생기면 식도가 막히나요?,0
4,"인문과학 또는 인문학(人文學, 영어: humanities)은 인간과 인간의 근원문제...",인문과학은 경험적인 접근을 주로 사용하는가?,0


In [ ]:
kobest_df.to_csv("data/kobest/kobest.csv", index=False)

In [ ]:
generation_prompt = PromptTemplate.from_template(
    load_template(file_name="question_choices_generation.txt", template_type="question_choices_generation")
)

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    api_key=os.getenv("OPENAI_API_KEY"),
)

output_parser = JsonOutputParser()

fixing_parser = OutputFixingParser.from_llm(parser=output_parser, llm=llm)

problem_generator = generation_prompt | llm | fixing_parser

In [ ]:
def generate_problem(data: pd.Series):
    response = problem_generator.invoke({PARAGRAPH: data[PARAGRAPH]})
    return response[QUESTION], response[CHOICES], response[ANSWER]


tqdm.pandas()
sampled_df = kobest_df.sample(20, random_state=1004)
sampled_df["result"] = sampled_df.progress_apply(generate_problem, axis=1)

# 결과 분리
sampled_df[QUESTION] = sampled_df["result"].apply(lambda x: x[0])
sampled_df[CHOICES] = sampled_df["result"].apply(lambda x: x[1])
sampled_df[ANSWER] = sampled_df["result"].apply(lambda x: x[2])

# 임시 열 삭제
sampled_df.drop(columns=["result", "label"], inplace=True)

sampled_df.head()

 20%|██        | 4/20 [00:08<00:35,  2.23s/it]

In [ ]:
sampled_df.to_csv("data/experiments/kobest_exp_v4.0.1.csv", index=False)

In [ ]:
validation_prompt = PromptTemplate(
    load_template(file_name="validation.txt", template_type="question_choices_generation")
)

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    api_key=os.getenv("OPENAI_API_KEY"),
)

output_parser = BooleanOutputParser()

fixing_parser = OutputFixingParser.from_llm(parser=output_parser, llm=llm)

validator = validation_prompt | llm | fixing_parser

In [ ]:
def validate_problem(data: pd.Series):
    response = validator.invoke(
        {PARAGRAPH: data[PARAGRAPH], QUESTION: data[QUESTION], CHOICES: data[CHOICES], ANSWER: data[ANSWER]}
    )
    return response


tqdm.pandas()
sampled_df = kobest_df.sample(20, random_state=1004)
sampled_df[VALID] = sampled_df.progress_apply(generate_problem, axis=1)

sampled_df.head()

In [ ]:
sampled_df.to_csv("data/experiments/kobest_exp_v4.0.1.csv", index=False)